# Machine Learning Model Serving for the Masses 
## Creating SQL UDFs in Unity Catalog for Use Reporting & Dashboards

In [0]:
SELECT
  ai_query(
    'patient_readmission_endpoint',
    request =>
      named_struct(
        "MARITAL_M", 0,
        "MARITAL_S", 1,
        "RACE_asian", 0,
        "RACE_black", 0,
        "RACE_hawaiian", 0,
        "RACE_other", 0,
        "RACE_white", 1,
        "ETHNICITY_hispanic", 0,
        "ETHNICITY_nonhispanic", 1,
        "GENDER_F", 0,
        "GENDER_M", 1,
        "INCOME", 30440,
        "BASE_ENCOUNTER_COST", 85.55,
        "TOTAL_CLAIM_COST", 802.11,
        "PAYER_COVERAGE", 0,
        "enc_length", 2908,
        "ENCOUNTERCLASS_ambulatory", 0,
        "ENCOUNTERCLASS_emergency", 0,
        "ENCOUNTERCLASS_hospice", 0,
        "ENCOUNTERCLASS_inpatient", 0,
        "ENCOUNTERCLASS_outpatient", 1,
        "ENCOUNTERCLASS_wellness", 0,
        "age_at_encounter", 41.530458590006845
      )
      ,returnType => 'float'
  ) as patient_readmission_risk

In [0]:
USE mgiglia.hls_genie;

In [0]:
SHOW TABLES;

In [0]:
SELECT DISTINCT
  p.patient_id
  ,p.birth_date
  ,p.gender
  ,p.ethnicity
  ,p.marital
  ,p.race
  ,p.income
  ,e.start as enc_start
  ,e.stop as enc_stop
  ,e.total_claim_cost
  ,e.payer_coverage
  ,e.encounter_class
FROM 
  patients as p
  ,encounters as e JOIN payers ON e.patient_id = p.patient_id
;

In [0]:
CREATE OR REPLACE FUNCTION age_at_encounter(
  birth_date DATE COMMENT 'Patient birth date'
  ,enc_date DATE COMMENT 'Encounter date or encounter start date'
) 
RETURNS DOUBLE COMMENT 'Age at encounter'
RETURN 
  DATEDIFF(enc_date, birth_date) / 365.25;

In [0]:
select 
  p.patient_id
  ,p.birth_date
  ,e.start
  ,age_at_encounter(p.birth_date, e.start) as age_at_encounter 
from 
  patients as p join 
  encounters as e on p.patient_id = e.patient_id
;

In [0]:
DROP FUNCTION race_one_hot;

In [0]:
CREATE OR REPLACE FUNCTION race_one_hot(
  race STRING COMMENT 'Patient race'
)
RETURNS STRUCT<
  RACE_asian INTEGER,
  RACE_black INTEGER,
  RACE_hawaiian INTEGER,
  RACE_other INTEGER,
  RACE_white INTEGER
> COMMENT 'One hot encoded race'
RETURN STRUCT(
  CASE WHEN race = 'asian' THEN 1 ELSE 0 END AS RACE_asian,
  CASE WHEN race = 'black' THEN 1 ELSE 0 END AS RACE_black,
  CASE WHEN race = 'hawaiian' THEN 1 ELSE 0 END AS RACE_hawaiian,
  CASE WHEN race = 'other' THEN 1 ELSE 0 END AS RACE_other,
  CASE WHEN race = 'white' THEN 1 ELSE 0 END AS RACE_white
)
;

In [0]:
SELECT
  p.patient_id
  ,p.race
  ,race_one_hot(p.race) as race_one_hot
FROM patients p

In [0]:
CREATE OR REPLACE FUNCTION marital_one_hot(
  marital STRING COMMENT 'Patient marital status'
)
RETURNS STRUCT<marital_s: INT, marital_m: INT> COMMENT 'One hot encoded marital status'
RETURN named_struct(
  'MARITAL_S', CASE WHEN marital = 'S' THEN 1 ELSE 0 END,
  'MARITAL_M', CASE WHEN marital = 'M' THEN 1 ELSE 0 END
);

In [0]:
SELECT 
  marital
  ,marital_one_hot(marital) as marital_one_hot
FROM patients;

In [0]:
CREATE OR REPLACE FUNCTION ethnicity_one_hot(ethnicity STRING)
RETURNS STRUCT<
  ETHNICITY_hispanic: INT, 
  ETHNICITY_nonhispanic: INT
>
RETURN 
  CASE 
    WHEN ethnicity = 'hispanic' THEN named_struct('ETHNICITY_hispanic', 1, 'ETHNICITY_nonhispanic', 0)
    ELSE named_struct('ETHNICITY_hispanic', 0, 'ETHNICITY_nonhispanic', 1)
  END;

In [0]:
select
  ethnicity
  ,ethnicity_one_hot(ethnicity) as ethnicity_one_hot
from patients

In [0]:
create or replace function gender_one_hot(gender string)
returns struct<GENDER_F: int, GENDER_M: int>
return named_struct(
  'GENDER_F', case when gender = 'F' then 1 else 0 end,
  'GENDER_M', case when gender = 'M' then 1 else 0 end
)

In [0]:
SELECT 
  gender
  ,gender_one_hot(gender)
FROM patients;

In [0]:
CREATE OR REPLACE FUNCTION encounterclass_one_hot(encounterclass STRING)
RETURNS STRUCT<
  ENCOUNTERCLASS_ambulatory INT,
  ENCOUNTERCLASS_emergency INT,
  ENCOUNTERCLASS_hospice INT,
  ENCOUNTERCLASS_inpatient INT,
  ENCOUNTERCLASS_outpatient INT,
  ENCOUNTERCLASS_wellness INT
>
RETURN (
  CASE encounterclass
    WHEN 'ambulatory' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 1, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'emergency' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 1, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'hospice' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 1, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'inpatient' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 1, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'outpatient' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 1, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'wellness' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 1)
    ELSE NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
  END
);

In [0]:
select 
  encounter_class
  ,encounterclass_one_hot(encounter_class) as encounterclass_one_hot
from encounters;

In [0]:
CREATE OR REPLACE FUNCTION encouter_length (
  start_date DATE
  ,end_date DATE
)
RETURNS INTEGER COMMENT 'Length of encounter in days'
RETURN datediff(end_date, start_date);

In [0]:
SELECT 
  start
  ,stop
  ,encouter_length(start, stop) as length
from encounters

In [0]:
DROP FUNCTION IF EXISTS create_readmit_struct;

In [0]:
CREATE OR REPLACE FUNCTION create_readmit_struct(
  birth_date DATE,
  gender STRING,
  ethnicity STRING,
  marital STRING,
  race STRING,
  income INT,
  enc_start TIMESTAMP,
  enc_stop TIMESTAMP,
  base_encounter_cost DOUBLE, 
  total_claim_cost DOUBLE,
  payer_coverage DOUBLE,
  encounter_class STRING
)
RETURNS STRUCT<
  MARITAL_S LONG,
  MARITAL_M LONG,
  RACE_asian LONG,
  RACE_black LONG,
  RACE_hawaiian LONG,
  RACE_other LONG,
  RACE_white LONG,
  GENDER_F LONG,
  GENDER_M LONG, 
  INCOME LONG, 
  BASE_ENCOUNTER_COST LONG,
  TOTAL_CLAIM_COST DOUBLE,
  PAYER_COVERAGE DOUBLE,
  enc_length LONG,
  ETHNICITY_hispanic LONG, 
  ETHNICITY_nonhispanic LONG,
  ENCOUNTERCLASS_ambulatory LONG,
  ENCOUNTERCLASS_emergency LONG,
  ENCOUNTERCLASS_hospice LONG,
  ENCOUNTERCLASS_inpatient LONG,
  ENCOUNTERCLASS_outpatient LONG,
  ENCOUNTERCLASS_wellness LONG,
  age_at_encounter DOUBLE 
>
RETURN named_struct(
  'MARITAL_S', marital_one_hot(marital).MARITAL_S,
  'MARITAL_M', marital_one_hot(marital).MARITAL_M,
  'RACE_asian', race_one_hot(race).RACE_asian,
  'RACE_black', race_one_hot(race).RACE_black,
  'RACE_hawaiian', race_one_hot(race).RACE_hawaiian,
  'RACE_other', race_one_hot(race).RACE_other,
  'RACE_white', race_one_hot(race).RACE_white,
  'GENDER_F', gender_one_hot(gender).GENDER_F,
  'GENDER_M', gender_one_hot(gender).GENDER_M,
  'INCOME', income,
  'BASE_ENCOUNTER_COST', base_encounter_cost,
  'TOTAL_CLAIM_COST', total_claim_cost,
  'PAYER_COVERAGE', payer_coverage,
  'enc_length', encouter_length(enc_start, enc_stop),
  'ETHNICITY_hispanic', ethnicity_one_hot(ethnicity).ETHNICITY_hispanic,
  'ETHNICITY_nonhispanic', ethnicity_one_hot(ethnicity).ETHNICITY_nonhispanic,
  'ENCOUNTERCLASS_ambulatory', encounterclass_one_hot(encounter_class).ENCOUNTERCLASS_ambulatory,
  'ENCOUNTERCLASS_emergency', encounterclass_one_hot(encounter_class).ENCOUNTERCLASS_emergency,
  'ENCOUNTERCLASS_hospice', encounterclass_one_hot(encounter_class).ENCOUNTERCLASS_hospice,
  'ENCOUNTERCLASS_inpatient', encounterclass_one_hot(encounter_class).ENCOUNTERCLASS_inpatient,
  'ENCOUNTERCLASS_outpatient', encounterclass_one_hot(encounter_class).ENCOUNTERCLASS_outpatient,
  'ENCOUNTERCLASS_wellness', encounterclass_one_hot(encounter_class).ENCOUNTERCLASS_wellness,
  'age_at_encounter', age_at_encounter(birth_date, enc_start)
)

In [0]:
select 
  p.patient_id
  ,create_readmit_struct(
    p.birth_date
    ,p.gender
    ,p.ethnicity
    ,p.marital
    ,p.race
    ,p.income
    ,e.start
    ,e.stop
    ,e.base_encounter_cost
    ,e.total_claim_cost
    ,e.payer_coverage
    ,e.encounter_class
  )
FROM 
  patients as p
  ,encounters as e JOIN payers ON e.patient_id = p.patient_id
;


In [0]:
DROP FUNCTION IF EXISTS patient_readmission_risk;

In [0]:
CREATE OR REPLACE FUNCTION patient_readmission_risk (
  birth_date DATE,
  gender STRING,
  ethnicity STRING,
  marital STRING,
  race STRING,
  income INT,
  enc_start TIMESTAMP,
  enc_stop TIMESTAMP,
  base_encounter_cost DOUBLE,
  total_claim_cost DOUBLE,
  payer_coverage DOUBLE,
  encounter_class STRING
)
RETURNS FLOAT COMMENT 'Predicted risk of readmission, 1 is high risk, 0 is low risk.'
RETURN
  ai_query(
    'patient_readmission_endpoint'
    ,request => create_readmit_struct(
      birth_date
      ,gender
      ,ethnicity
      ,marital
      ,race
      ,income
      ,enc_start
      ,enc_stop
      ,base_encounter_cost
      ,total_claim_cost
      ,payer_coverage
      ,encounter_class
    )
    ,returnType => 'float'
  )
;

In [0]:
SELECT 
  p.patient_id
  ,e.encounter_id
  ,patient_readmission_risk(
    p.birth_date
    ,p.gender
    ,p.ethnicity
    ,p.marital
    ,p.race
    ,p.income
    ,e.start
    ,e.stop
    ,e.base_encounter_cost
    ,e.total_claim_cost
    ,e.payer_coverage
    ,e.encounter_class
  ) as patient_readmission_risk
FROM 
  patients as p INNER JOIN encounters as e ON p.patient_id = e.patient_id
;